# Read a MySQL DataBase and generate specific output

In [32]:
import getpass
import pymysql
import unidecode
import pandas as pd
import tempfile
import subprocess
import sys

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth',500)

def generate_pdf(row):
    
    if row.opening is  None:
        row.opening=''
    if row.closing is  None:
        row.closing=''

    latexfile='\n'
    latexfile=latexfile+'%'+row.description+'\n'
    latexfile=latexfile+r'\documentclass{letter}'+'\n'
    latexfile=latexfile+r'\usepackage[utf8]{inputenc}'+'\n'
    latexfile=latexfile+row.preamble+'\n'
    latexfile=latexfile+row.address+'\n'
    latexfile=latexfile+r'\signature{'+row.sign+'}\n'
    latexfile=latexfile+r'\begin{document}'+'\n'
    latexfile=latexfile+r'\begin{letter}{'+row.para+'}\n'
    latexfile=latexfile+r'\opening{'+row.opening+'}\n'
    latexfile=latexfile+row.body+'\n'
    latexfile=latexfile+r'\closing{'+row.closing+'}\n'
    latexfile=latexfile+r'\end{letter}'+'\n'
    latexfile=latexfile+r'\end{document}'+'\n'

    f=tempfile.NamedTemporaryFile('w',suffix='.tex',delete=False)
    f.write(latexfile)
    f.close()
    lo=subprocess.Popen('pdflatex {:s}'.format(f.name).split(),cwd='/tmp',
                        stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()
    pdffile='{:s}'.format(f.name).split('.tex')[0]+'.pdf'
    if lo[0].decode('utf8').find(pdffile.split('/')[-1])>-1:
        pdf='carta_{:d}.pdf'.format(row['id'])
        dwld_dir='downloads'
        kk=subprocess.Popen('mv {:s} /home/restrepo/{:s}/{:s}'.format(pdffile,dwld_dir,pdf).split()).communicate()
        print('output in http://fisica.udea.edu.co/{:s}/{:s}'.format(dwld_dir,pdf))
        for suffix in  ['.aux','.log','.out','.tex']:
            sc=subprocess.Popen( ['rm', '{:s}'.format(f.name).split('.tex')[0]+suffix] ).communicate()
    else:
        print(lo[0].decode('utf8'))
        sys.exit('LaTeX ERROR:')    
    return latexfile

def db_to_pdf(choose_db=0,id=None,verbose=False):
    '''
    Obtain letter from the last DB entry
    Type of letter: 
      * 0 Personal
      * 1 Group
    '''
    password=getpass.getpass()
    tipo_carta=['Diego_Restrepo', 'cartas_gfif']
    conn = pymysql.connect(host='localhost', port=3306, user='root', passwd=password, db=tipo_carta[choose_db])

    df=pd.read_sql('select * from cartas;', con=conn)

    sg=pd.read_sql('select * from cartas_sign;', con=conn)
    
    
    conn.close()
    
    #print(df.shape[0])
    #return df,sg
    if df.shape[0]:
        if not id:
            id=df.id.values[-1]
        c=df[df.id==id].reset_index(drop=True).loc[0]
        c['sign']=sg[sg.signature==c.signature].reset_index(drop=True).loc[0,'sign']
        if verbose:
            print(c)
    else:
        sys.exit('Error reading databases')
        return df

    
    
    return generate_pdf(c)

def backup_database(choose_db=0):
    '''
    Obtain letter from the last DB entry
    Type of letter: 
      * 0 Personal
      * 1 Group
    '''
    password=getpass.getpass()
    tipo_carta=['Diego_Restrepo', 'cartas_gfif']
    
    print('Trying to create backup file db_{:s}_backup.sql'.format( tipo_carta[choose_db]  )   )
    ok=subprocess.Popen('mysqldump -u root -p{:s} {:s} > db_{:s}_backup.sql'.format(
                     password,tipo_carta[choose_db],tipo_carta[choose_db]),shell=True,
                     stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()
    
    print()
    
    return ok

In [37]:
choose_db=0
latexfile=db_to_pdf(choose_db,id=284)

········
output in http://fisica.udea.edu.co/downloads/carta_284.pdf


In [3]:
print(latexfile)

NameError: name 'latexfile' is not defined

### Backup database

In [34]:
backup_database(choose_db)

········
Trying to create backup file db_Diego_Restrepo_backup.sql



(b'', b'')

## Manual

In [4]:
password=getpass.getpass()

········


In [11]:
choose_db=0
tipo_carta=['Diego_Restrepo', 'cartas_gfif']

conn = pymysql.connect(host='localhost', port=3306, user='root', passwd=password, db=tipo_carta[choose_db])

df=pd.read_sql('select * from cartas;', con=conn)

sg=pd.read_sql('select * from cartas_sign;', con=conn)

In [12]:
conn.close()

In [7]:
df=df[df.body.str.lower().str.contains('editor')]

In [13]:
df=df[df.id==49]

Last entry

In [14]:
df

,id,description,preamble,para,address,signature,opening,body,closing,fecha,link
46,49,Carta de Invitación,\usepackage{udea}\r\n\def{\fullname}{Dr. Marcos Cardoso Rodriguez}\r\n\def\lastname{Rodriguez},\fullname\\\r\nGrupo de Física Teórica e Matemática\\\r\nDepartamento de Física\\\r\nUniversidade Federal Rural do Rio de Janeiro\r\n,,Diego_en_sign,None,"I have the pleasure to invite \fullname\ for a\r\nvisit to our research group at Insituto de Física, Universidad de\r\nAntioquia from November 27 to December 1 of present year, in order to continue\r\nwith research works that we are developing within the framework of our\r\ncollaboration and to give a course on neutrino masses in SUSY\r\n%It pleases to us to say that all %the expenses caused by\r\n%his trip and stay in Colombia %will be covered by my group.\r\n\r\nI beforehand thanks all t...",Best regards,2007-01-31,http://naima.udea.edu.co/cartas/cartas.php?step=1&id=49


In [15]:
c=df[df.id==df.id.values[-1]].reset_index(drop=True).loc[0]
c['sign']=sg[sg.signature==c.signature].reset_index(drop=True).loc[0,'sign']

In [16]:
c

id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              49
description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [17]:
row=c

In [31]:
lo=generate_pdf(row)

This is pdfTeX, Version 3.14159265-2.6-1.40.15 (TeX Live 2015/dev/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(/tmp/tmpg_gp_7ls.tex
LaTeX2e <2014/05/01>
Babel <3.9l> and hyphenation patterns for 79 languages loaded.
(/usr/share/texlive/texmf-dist/tex/latex/base/letter.cls
Document Class: letter 2014/09/29 v1.2z Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/base/inputenc.sty
(/usr/share/texlive/texmf-dist/tex/latex/base/utf8.def
(/usr/share/texlive/texmf-dist/tex/latex/base/t1enc.dfu)
(/usr/share/texlive/texmf-dist/tex/latex/base/ot1enc.dfu)
(/usr/share/texlive/texmf-dist/tex/latex/base/omsenc.dfu)))
(/usr/share/texlive/texmf-dist/tex/latex/cartas/udea.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/graphicx.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/share/texlive/texmf-dist/tex/latex/graphics/graphics.sty
(/usr/share/texlive

SystemExit: LaTeX ERROR:

/usr/local/lib/python3.4/dist-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [29]:
print(lo[0].decode('utf8'))

This is pdfTeX, Version 3.14159265-2.6-1.40.15 (TeX Live 2015/dev/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(/tmp/tmpj2_wegy8.tex
LaTeX2e <2014/05/01>
Babel <3.9l> and hyphenation patterns for 79 languages loaded.
(/usr/share/texlive/texmf-dist/tex/latex/base/letter.cls
Document Class: letter 2014/09/29 v1.2z Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/base/inputenc.sty
(/usr/share/texlive/texmf-dist/tex/latex/base/utf8.def
(/usr/share/texlive/texmf-dist/tex/latex/base/t1enc.dfu)
(/usr/share/texlive/texmf-dist/tex/latex/base/ot1enc.dfu)
(/usr/share/texlive/texmf-dist/tex/latex/base/omsenc.dfu)))
(/usr/share/texlive/texmf-dist/tex/latex/cartas/udea.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/graphicx.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/share/texlive/texmf-dist/tex/latex/graphics/graphics.sty
(/usr/share/texlive

In [21]:
c.sign

"\\includegraphics[scale=0.4]{firma}\\\\\r\n\\noindent\r\nDiego A. Restrepo Quintero\\\\\r\nCoordinador\\\\\r\nGrupo de Fenomenología de Interacciones Fundamentales\\\\\r\nInstituto de F\\'\\i sica\\\\\r\nUniversidad de Antioquia"

In [18]:
ok

(b'', b'')

In [10]:
'mysqldump -u root -p{:s} {:s} > db_{:s}_backup.sql'.format(password,tipo_carta[NTC],tipo_carta[NTC]).split()

['mysqldump',
 '-u',
 'root',
 '-ppoioiulkj',
 'cartas_gfif',
 '>',
 'db_cartas_gfif_backup.sql']

## Search in body

In [50]:
id=99
if id >0:
    dfs=df[df.id==id].reset_index(drop=True)
else:    
    dfs=df[df.body.str.lower().map(unidecode.unidecode).str.contains('impacto social')].reset_index(drop=True)

In [ ]:
dfs

In [52]:
c=dfs.loc[dfs.index.values[-1]]

In [ ]:
c

In [ ]:
c.keys().values

In [ ]:
for k in c.keys():
    print(k)
    if c[k]==None:
        c[k]=''

In [52]:
c.closing

In [34]:
c.closing

'Best regards'

In [35]:
latexfile=r'''
%'''+c.description+r'''
\documentclass{letter}
\usepackage[utf8]{inputenc}
'''+c.preamble+r'''
'''+c.address+r'''
\signature{'''+sg[sg.signature==c.signature].sign.values[0]+r'''}
\begin{document}
\begin{letter}{'''+c.para+r'''}

\opening{'''+c.opening+r'''} 

'''+c.body+r'''

\closing{'''+c.closing+r'''}
\end{letter}
\end{document}
'''

TypeError: Can't convert 'NoneType' object to str implicitly

In [ ]:
print(latexfile)

In [58]:
f=tempfile.NamedTemporaryFile('w',suffix='.tex',delete=False)

In [59]:
f.write(latexfile)

2017

In [60]:
f.name

'/tmp/tmppt2pwnu1.tex'

In [61]:
f.close()

In [62]:
lo=subprocess.Popen('pdflatex {:s}'.format(f.name).split(),cwd='/tmp',stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()

In [ ]:
pdffile='{:s}'.format(f.name).split('.tex')[0]+'.pdf'
if lo[0].decode('utf8').find(pdffile.split('/')[-1])>-1:
    pdf='carta_{:d}.pdf'.format(c['id'])
    dwld_dir='downloads'
    kk=subprocess.Popen('mv {:s} /var/www/{:s}/{:s}'.format(pdffile,dwld_dir,pdf).split()).communicate()
    print('output in http://gfif.udea.edu.co/{:s}/{:s}'.format(dwld_dir,pdf))
    for suffix in  ['.aux','.log','.out','.tex']:
        subprocess.Popen( ['rm', '{:s}'.format(f.name).split('.tex')[0]+suffix] ).communicate()
else:
    sys.exit('LaTeX ERROR:')    